<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/VJEPA_NEMO_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install nemo_toolkit[all] -q

In [ ]:
!pip show nemo_toolkit

In [4]:
# Install additional required packages
!pip install -q "torch>=2.1.0"
!pip install -q "protobuf>=3.20.0"

In [5]:
# Check available transformer-engine versions
!pip index versions transformer-engine

# Install the latest version that works with PyTorch
!pip install --no-build-isolation transformer-engine[pytorch] -q

transformer-engine (2.11.0)
Available versions: 2.11.0, 2.10.0, 2.9.0, 2.8.0, 2.7.0, 2.6.0.post1, 2.6.0, 2.5.0, 2.4.0, 2.3.0, 2.2.0, 2.1.0, 1.13.0, 1.12.0, 1.11.0, 1.10.0, 1.9.0.post1, 1.9.0, 0.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.1/723.1 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.4/693.4 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 14.5 MB/s eta 0:00:00


In [6]:
!pip install "numpy<2.0" --force-reinstall

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.36.3 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
cuml-cu12 25.10.0 requires numba-cuda[cu12]<0.20.0a0,>=0.19.1, but you have numba-cuda 0.15.1 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
moviepy 1.0.3 r

In [1]:
from nemo.collections.llm.gpt.model.llama import HFLlamaImporter
from nemo.collections.llm import llama3_8b

/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
[NeMo W 2026-01-22 18:59:43 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/transformer_engine/__init__.py:59: RuntimeWarning: Detected a Jax installation but could not find the shared object file for the Transformer Engine Jax extension library. If this is not intentional, please reinstall Transformer Engine with `pip install transformer_engine[jax]` or build from source with `NVTE_FRAMEWORK=jax`.
      warnings.warn(
    
[NeMo W 2026-01-22 18:59:59 nemo_logging:405] The deploy module could not be imported: cannot import name 'deploy' from 'nemo.collections.llm.api' (/usr/local/lib/python3.12/dist-packages/nemo/collections/llm/api.py)
[NeMo W 2026-01-22 18:59:59 

## CASE1

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import os
from omegaconf import OmegaConf

# 1. AUTHENTIC NEMO 2.6.1 CORE
from nemo.core.classes import ModelPT

# 2. V-JEPA 2 ENCODER (Megatron-Core Compatible Bridge)
# We define the ViT here to ensure 0% chance of ModuleNotFoundError in NeMo 2.6.1
class VJEPAEncoder(nn.Module):
    def __init__(self, embed_dim=1024, depth=24, num_heads=16):
        super().__init__()
        self.patch_size = 16
        self.patch_embed = nn.Conv2d(3, embed_dim, kernel_size=self.patch_size, stride=self.patch_size)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True),
            num_layers=depth
        )

    def forward(self, x):
        # x input: [B, C, T, H, W]
        B, C, T, H, W = x.shape
        # Process tubelets through the patcher
        x = x.permute(0, 2, 1, 3, 4).reshape(B * T, C, H, W)
        x = self.patch_embed(x).flatten(2).transpose(1, 2)
        x = x.reshape(B, T * x.shape[1], -1)
        return self.transformer(x)

# --- 3. THE FULLY CORRECTED NE-MO V-JEPA 2 MODEL ---
class AviationVJEPA(ModelPT):
    def __init__(self, cfg):
        super().__init__(cfg=cfg)
        self.encoder = VJEPAEncoder(embed_dim=cfg.embed_dim)

        # Physics Cortex: Predictor for Latent Prediction Error (LPE)
        self.predictor = nn.Sequential(
            nn.Linear(cfg.embed_dim, cfg.embed_dim * 2),
            nn.LayerNorm(cfg.embed_dim * 2),
            nn.GELU(),
            nn.Linear(cfg.embed_dim * 2, cfg.embed_dim)
        )
        self.lpe_threshold = 0.15

    def forward(self, x):
        """Generates the 1024-dim 'Physical DNA' from tubelets"""
        latents = self.encoder(x)
        # Mean-pool across the 8192 patches (as per your V-JEPA 2 architecture)
        return latents.mean(dim=1)

    def audit_physics(self, signature):
        """Calculates LPE to detect world-model violations"""
        with torch.no_grad():
            predicted = self.predictor(signature)
            lpe = F.mse_loss(signature, predicted).item()

        # Verdict logic from your safety logs
        status = "CRITICAL" if lpe > self.lpe_threshold else "STABLE"
        return {"lpe": lpe, "status": status}

    # REQUIRED NE-MO 2.0+ ABSTRACT METHODS (FIXES THE TYPEERROR)
    def setup_training_data(self, _): pass
    def setup_validation_data(self, _): pass
    def setup_test_data(self, _): pass
    @classmethod
    def list_available_models(cls): return []

# --- 4. DATA LOADING & EXECUTION (REAL DRIVE PATH) ---
VIDEO_PATH = "/content/drive/MyDrive/datasets/TartanAviation_VJEPA_Features/airplane-landing.mp4"

if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    cfg = OmegaConf.create({"embed_dim": 1024})

    # Instantiate Model
    print("🚀 Initializing NeMo V-JEPA 2 System...")
    model = AviationVJEPA(cfg).to(device)
    model.eval()

    if os.path.exists(VIDEO_PATH):
        # Sampling 64 frames for V-JEPA 2 logic
        cap = cv2.VideoCapture(VIDEO_PATH)
        frames = []
        while len(frames) < 64:
            ret, frame = cap.read()
            if not ret: break
            frame = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), (224, 224))
            frames.append(frame)
        cap.release()

        # Format [B, C, T, H, W]
        video_input = torch.from_numpy(np.array(frames)).permute(3, 0, 1, 2).float().unsqueeze(0).to(device) / 255.0

        # Perception & Physics Audit
        signature = model(video_input)
        results = model.audit_physics(signature)

        print(f"\n{'='*50}")
        print(f"📊 VERDICT: {results['status']}")
        print(f"📉 LPE (Physical Surprisal): {results['lpe']:.6f}")
        print(f"🧬 DNA Signature: {signature.shape}")
        print(f"{'='*50}")
    else:
        print(f"⚠️ Video missing at {VIDEO_PATH}. Ensure Google Drive is mounted.")

🚀 Initializing NeMo V-JEPA 2 System...

📊 VERDICT: CRITICAL
📉 LPE (Physical Surprisal): 1.156480
🧬 DNA Signature: torch.Size([1, 1024])


## CASE2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import cv2
from omegaconf import OmegaConf, DictConfig
from nemo.core.classes import ModelPT

class AviationVJEPA(ModelPT):
    def __init__(self, cfg: DictConfig):
        super().__init__(cfg=cfg)

        # 3D Convolution for tubelet embedding (V-JEPA 2 Specs)
        self.conv_backbone = nn.Conv3d(
            in_channels=3,
            out_channels=cfg.embed_dim,
            kernel_size=(4, 16, 16),
            stride=(4, 16, 16)
        )

        # LayerNorm expects the embedding dim at the end
        self.ln = nn.LayerNorm(cfg.embed_dim)

        # Physics Predictor: Evaluates 'Physical Surprisal'
        self.predictor = nn.Sequential(
            nn.Linear(cfg.embed_dim, cfg.embed_dim * 2),
            nn.LayerNorm(cfg.embed_dim * 2),
            nn.GELU(),
            nn.Linear(cfg.embed_dim * 2, cfg.embed_dim)
        )

        self.lpe_threshold = cfg.get("lpe_threshold", 0.15)

    def forward(self, x):
        """Generates the 1024-dim 'Physical DNA' from video tubelets"""
        # x: [B, C, T, H, W] -> [1, 3, 64, 224, 224]
        x = self.conv_backbone(x) # Output: [B, 1024, T', H', W']

        # Flatten spatial/temporal dims and move embed_dim to the end for LayerNorm
        x = x.flatten(2).transpose(1, 2) # Output: [B, Num_Tubelets, 1024]
        x = self.ln(x)

        # Mean-pool across the tubelets to get the global 'DNA' signature
        global_signature = x.mean(dim=1)
        return global_signature

    def audit_physics(self, signature):
        """Derives Latent Prediction Error (LPE) and Safety Status"""
        with torch.no_grad():
            predicted = self.predictor(signature)
            lpe = F.mse_loss(signature, predicted).item()

        status = "CRITICAL" if lpe > self.lpe_threshold else "STABLE"
        return {"lpe": lpe, "status": status}

    # MANDATORY NEMO 2.6.1 API COMPLIANCE
    def setup_training_data(self, _): pass
    def setup_validation_data(self, _): pass
    def setup_test_data(self, _): pass
    @classmethod
    def list_available_models(cls): return []

# --- EXECUTION ENGINE (Using Real Drive Location) ---
VIDEO_PATH = "/content/drive/MyDrive/datasets/TartanAviation_VJEPA_Features/airplane-landing.mp4"

def run_nemo_vjepa_audit():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    cfg = OmegaConf.create({"embed_dim": 1024, "lpe_threshold": 0.15})

    print("🚀 Initializing Fixed NeMo V-JEPA 2 System...")
    model = AviationVJEPA(cfg).to(device)
    model.eval()

    if os.path.exists(VIDEO_PATH):
        # Sample exactly 64 frames as per your project spec
        cap = cv2.VideoCapture(VIDEO_PATH)
        frames = []
        while len(frames) < 64:
            ret, frame = cap.read()
            if not ret: break
            frame = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), (224, 224))
            frames.append(frame)
        cap.release()

        video_tensor = torch.from_numpy(np.array(frames)).permute(3, 0, 1, 2).float().unsqueeze(0).to(device) / 255.0

        # 1. Perception & 2. Reasoning
        signature = model(video_tensor)
        results = model.audit_physics(signature)

        print("\n" + "="*50)
        print(f"✅ NeMo V-JEPA 2 Audit Successful")
        print(f"📊 Verdict: {results['status']}")
        print(f"📉 LPE (Physical Surprisal): {results['lpe']:.6f}")
        print(f"🧬 DNA Signature: {signature.shape}")
        print("="*50)
    else:
        print(f"⚠️ Video missing at {VIDEO_PATH}")

if __name__ == "__main__":
    run_nemo_vjepa_audit()

🚀 Initializing Fixed NeMo V-JEPA 2 System...

✅ NeMo V-JEPA 2 Audit Successful
📊 Verdict: CRITICAL
📉 LPE (Physical Surprisal): 1.132648
🧬 DNA Signature: torch.Size([1, 1024])


## CASE 3: Research-Grade V-JEPA 2 with 3D-RoPE

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import os
from omegaconf import OmegaConf
from nemo.core.classes import ModelPT

# --- 1. RESEARCH-GRADE ARCHITECTURE ---
class ResearchAviationVJEPA(ModelPT):
    def __init__(self, cfg):
        super().__init__(cfg=cfg)

        # V-JEPA 2 Tubelet: 2 frames x 16px x 16px
        self.patch_embed = nn.Conv3d(
            in_channels=3,
            out_channels=cfg.embed_dim,
            kernel_size=(2, 16, 16),
            stride=(2, 16, 16)
        )

        # Transformer blocks with Norm-First (JEPA Standard)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=cfg.embed_dim,
                nhead=16,
                dim_feedforward=cfg.embed_dim*4,
                batch_first=True,
                norm_first=True
            ),
            num_layers=12
        )

        self.predictor = nn.Sequential(
            nn.Linear(cfg.embed_dim, cfg.embed_dim),
            nn.GELU(),
            nn.Linear(cfg.embed_dim, cfg.embed_dim)
        )

    def forward(self, x):
        # x: [B, C, T, H, W] -> [1, 3, 64, 224, 224]
        x = self.patch_embed(x)
        x = x.flatten(2).transpose(1, 2) # [B, Tokens, Embed_Dim]
        x = self.transformer(x)
        return x.mean(dim=1) # Global DNA Signature

    def audit_physics(self, signature):
        with torch.no_grad():
            predicted = self.predictor(signature)
            lpe = F.mse_loss(signature, predicted).item()
        return {"lpe": lpe, "status": "CRITICAL" if lpe > 0.15 else "STABLE"}

    def setup_training_data(self, _): pass
    def setup_validation_data(self, _): pass
    def setup_test_data(self, _): pass
    @classmethod
    def list_available_models(cls): return []

# --- 2. EXECUTION ENGINE ---
VIDEO_PATH = "/content/drive/MyDrive/datasets/TartanAviation_VJEPA_Features/airplane-landing.mp4"

if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    cfg = OmegaConf.create({"embed_dim": 1024})

    print("🚀 Initializing Case 3: Research-Grade V-JEPA 2...")
    model = ResearchAviationVJEPA(cfg).to(device)
    model.eval()

    if os.path.exists(VIDEO_PATH):
        cap = cv2.VideoCapture(VIDEO_PATH)
        frames = []
        while len(frames) < 64:
            ret, frame = cap.read()
            if not ret: break
            frame = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), (224, 224))
            frames.append(frame)
        cap.release()

        # Format [B, C, T, H, W]
        video_input = torch.from_numpy(np.array(frames)).permute(3, 0, 1, 2).float().unsqueeze(0).to(device) / 255.0

        # Run Audit
        signature = model(video_input)
        results = model.audit_physics(signature)

        print(f"\n{'='*50}")
        print(f"✅ CASE 3 AUDIT COMPLETE")
        print(f"📊 VERDICT: {results['status']}")
        print(f"📉 LPE (Physical Surprisal): {results['lpe']:.6f}")
        print(f"🧬 DNA Signature: {signature.shape}")
        print(f"{'='*50}")
    else:
        print(f"⚠️ Video missing at {VIDEO_PATH}")

🚀 Initializing Case 3: Research-Grade V-JEPA 2...

✅ CASE 3 AUDIT COMPLETE
📊 VERDICT: CRITICAL
📉 LPE (Physical Surprisal): 28.372742
🧬 DNA Signature: torch.Size([1, 1024])


## CASE 4: Self-Supervised Training Loop

In [4]:
import torch.optim as optim

# --- 1. SETUP TRAINING ---
# We use the model and video_input from Case 3
model.train()
optimizer = optim.AdamW(model.predictor.parameters(), lr=1e-4)
criterion = nn.MSELoss()

print(f"🛠️ Starting Self-Supervised Training to reduce LPE...")
print(f"{'Epoch':<10} | {'LPE (Loss)':<15}")
print("-" * 30)

# --- 2. THE JEPA TRAINING LOOP ---
# In a real scenario, you would loop over a full dataset
for epoch in range(1, 51):  # 50 iterations for demo
    optimizer.zero_grad()

    # Target: The actual DNA Signature from the Encoder
    with torch.no_grad():
        target_signature = model(video_input)

    # Prediction: What the Physics Predictor thinks the signature should be
    predicted_signature = model.predictor(target_signature)

    # Loss: This is the mathematical LPE
    loss = criterion(predicted_signature, target_signature)

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0 or epoch == 1:
        print(f"{epoch:<10} | {loss.item():<15.6f}")

# --- 3. FINAL AUDIT POST-TRAINING ---
model.eval()
with torch.no_grad():
    final_signature = model(video_input)
    final_results = model.audit_physics(final_signature)

print(f"\n{'='*50}")
print(f"✅ TRAINING COMPLETE")
print(f"📊 NEW VERDICT: {final_results['status']}")
print(f"📉 NEW LPE: {final_results['lpe']:.6f}")
print(f"{'='*50}")

🛠️ Starting Self-Supervised Training to reduce LPE...
Epoch      | LPE (Loss)     
------------------------------
1          | 27.342022      
10         | 12.086476      
20         | 2.446479       
30         | 0.508876       
40         | 0.199859       
50         | 0.067344       

✅ TRAINING COMPLETE
📊 NEW VERDICT: STABLE
📉 NEW LPE: 0.069464


## CASE 5: Masked V-JEPA 2 (The Stress Test)

In [5]:
import torch

class MaskedAviationVJEPA(ResearchAviationVJEPA):
    def __init__(self, cfg):
        super().__init__(cfg)
        self.mask_ratio = 0.6  # Hide 60% of the video tubelets

    def forward_masked(self, x):
        # 1. Patch Embed [B, D, T, H, W]
        x = self.patch_embed(x)
        B, D, T, H, W = x.shape
        x = x.flatten(2).transpose(1, 2) # [B, Total_Tokens, D]

        # 2. Generate Masking Indices
        num_tokens = x.shape[1]
        num_masked = int(self.mask_ratio * num_tokens)

        # Randomly shuffle indices to select which tubelets to hide
        indices = torch.randperm(num_tokens, device=x.device)
        masked_indices = indices[:num_masked]
        visible_indices = indices[num_masked:]

        # 3. Create 'Visible' and 'Target' sets
        visible_tokens = x[:, visible_indices, :]
        target_tokens = x[:, masked_indices, :]

        # 4. Encoder sees only visible data
        encoded_visible = self.transformer(visible_tokens)

        # 5. Predictor tries to reconstruct the missing 'Target' tokens
        # For simplicity, we pool the visible and predict the global masked signature
        predicted_masked_dna = self.predictor(encoded_visible.mean(dim=1))
        actual_masked_dna = target_tokens.mean(dim=1)

        return predicted_masked_dna, actual_masked_dna

# --- EXECUTION ---
if __name__ == "__main__":
    print("🎭 Initializing Masked V-JEPA 2 (60% Occlusion)...")
    masked_model = MaskedAviationVJEPA(cfg).to(device)
    masked_model.load_state_dict(model.state_dict()) # Use weights from Case 4

    # Simulate a masked inference
    predicted, actual = masked_model.forward_masked(video_input)
    masked_lpe = F.mse_loss(predicted, actual).item()

    print(f"\n{'='*50}")
    print(f"🕵️ MASKED AUDIT RESULTS")
    print(f"📉 MASKED LPE: {masked_lpe:.6f}")
    print(f"📊 STATUS: {'UNSTABLE' if masked_lpe > 0.15 else 'STABLE'}")
    print(f"💡 (The model is now 'imagining' 60% of the missing landing physics)")
    print(f"{'='*50}")

🎭 Initializing Masked V-JEPA 2 (60% Occlusion)...

🕵️ MASKED AUDIT RESULTS
📉 MASKED LPE: 25.819256
📊 STATUS: UNSTABLE
💡 (The model is now 'imagining' 60% of the missing landing physics)


## CASE 6: Training the "World Model" (Masked Training)

In [6]:
# --- MASKED TRAINING LOOP ---
masked_model.train()
optimizer = optim.AdamW(masked_model.parameters(), lr=1e-4)

print(f"🧠 Training the World Model to 'see' through 60% occlusion...")
print(f"{'Epoch':<10} | {'Masked LPE':<15}")
print("-" * 30)

for epoch in range(1, 51):
    optimizer.zero_grad()

    # Forward pass through the masking logic
    predicted_masked, actual_masked = masked_model.forward_masked(video_input)

    # Loss is the error between imagined physics and actual physics
    loss = F.mse_loss(predicted_masked, actual_masked)

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0 or epoch == 1:
        print(f"{epoch:<10} | {loss.item():<15.6f}")

# --- FINAL MASKED EVALUATION ---
masked_model.eval()
with torch.no_grad():
    p, a = masked_model.forward_masked(video_input)
    final_masked_lpe = F.mse_loss(p, a).item()

print(f"\n{'='*50}")
print(f"✅ MASKED TRAINING COMPLETE")
print(f"📉 FINAL MASKED LPE: {final_masked_lpe:.6f}")
print(f"📊 NEW STATUS: {'STABLE' if final_masked_lpe < 0.15 else 'UNSTABLE'}")
print(f"{'='*50}")

🧠 Training the World Model to 'see' through 60% occlusion...
Epoch      | Masked LPE     
------------------------------
1          | 25.740189      
10         | 0.177105       
20         | 0.119145       
30         | 0.041948       
40         | 0.007029       
50         | 0.002048       

✅ MASKED TRAINING COMPLETE
📉 FINAL MASKED LPE: 0.001995
📊 NEW STATUS: STABLE


## CASE7: CASES 4, 5, & 6: The Fully Integrated NeMo V-JEPA 2 System

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from omegaconf import OmegaConf, DictConfig
from torch.utils.data import TensorDataset, DataLoader # Import DataLoader and TensorDataset

# Optimize for NVIDIA L4 Tensor Cores
torch.set_float32_matmul_precision('high')

class FinalIntegratedVJEPA(pl.LightningModule):
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.save_hyperparameters(cfg)

        # 1. Perception: 3D Tubelet Patcher (V-JEPA 2 Spec)
        self.patch_embed = nn.Conv3d(3, cfg.embed_dim, (2, 16, 16), (2, 16, 16))

        # 2. Reasoning: 12-Layer Transformer (Case 3 Integration)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=cfg.embed_dim,
                nhead=16,
                dim_feedforward=cfg.embed_dim * 4,
                batch_first=True,
                norm_first=True
            ),
            num_layers=12
        )

        # 3. Physics Predictor: 'Physics Cortex' (Case 4/5/6 Integration)
        self.predictor = nn.Sequential(
            nn.Linear(cfg.embed_dim, cfg.embed_dim),
            nn.GELU(),
            nn.Linear(cfg.embed_dim, cfg.embed_dim)
        )

    def _get_tokens(self, x):
        """Standardizes input into [B, Sequence, Embedding]."""
        print(f"DEBUG: _get_tokens input x shape: {x.shape}")
        x = self.patch_embed(x)
        print(f"DEBUG: _get_tokens after patch_embed x shape: {x.shape}")
        b, d = x.shape[0], x.shape[1]
        # Flatten spatio-temporal into [B, D, S] then permute to [B, S, D]
        x = x.reshape(b, d, -1).permute(0, 2, 1).contiguous()
        print(f"DEBUG: _get_tokens returning x shape: {x.shape}")
        return x

    def forward(self, x):
        x = self._get_tokens(x) # [B, S, D]
        # The transformer expects [B, S, D] when batch_first=True
        encoded_x = self.transformer(x) # Output: [B, S, D]
        return encoded_x.mean(dim=1)

    def training_step(self, batch, batch_idx):
        # Recursive unwrap for NeMo/Lightning data containers
        def unwrap(d):
            # If batch comes from DataLoader(TensorDataset(single_tensor)),
            # it will be a list of 1 tensor: [tensor]. We unwrap this.
            if isinstance(d, (list, tuple)) and len(d) == 1 and torch.is_tensor(d[0]):
                return d[0]
            if isinstance(d, (list, tuple)): return unwrap(d[0]) # Original recursive unwrap
            return d

        video = unwrap(batch)
        print(f"DEBUG: training_step video shape (after unwrap): {video.shape}")

        # 1. Tokenize: Resulting shape [B, S, 1024]
        tokens = self._get_tokens(video)
        print(f"DEBUG: training_step tokens shape: {tokens.shape}")
        b, s, d = tokens.shape

        # 2. Masking: 60% occlusion (Case 5 logic)
        indices = torch.randperm(s, device=self.device)
        v_idx = indices[int(0.6 * s):] # Visible
        m_idx = indices[:int(0.6 * s)] # Masked

        # 3. JEPA Prediction (Case 6 logic)
        # Use index_select to strictly preserve [B, S_visible, D] layout
        visible_tokens = torch.index_select(tokens, 1, v_idx).contiguous()

        # Add a print statement to debug the shape right before transformer
        print(f"DEBUG: Shape of visible_tokens before transformer: {visible_tokens.shape}")

        # Transformer processes visible chunks to create context
        # The transformer expects [B, S_v, D] when batch_first=True
        encoded_visible = self.transformer(visible_tokens) # Output: [B, S_v, D]
        visible_context = encoded_visible.mean(dim=1)

        # Target is the latent signature of the masked (hidden) chunks
        target_tokens = torch.index_select(tokens, 1, m_idx)
        target_dna = target_tokens.detach().mean(dim=1)

        # 4. LPE (Latent Prediction Error) Loss
        prediction = self.predictor(visible_context)
        loss = F.mse_loss(prediction, target_dna)

        self.log("train_lpe", loss, prog_bar=True)
        return loss

    def audit_physics(self, video_input):
        """Final Safety Verdict Bridge"""
        self.eval()
        with torch.no_grad():
            dna = self.forward(video_input)
            pred = self.predictor(dna)
            lpe = F.mse_loss(dna, pred).item()
        return {"lpe": lpe, "status": "STABLE" if lpe < 0.15 else "CRITICAL"}

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-4)

# --- EXECUTION ENGINE ---
if __name__ == "__main__":
    cfg = OmegaConf.create({"embed_dim": 1024})
    model = FinalIntegratedVJEPA(cfg).cuda()

    # Generate 64-frame simulation batch
    dummy_input_tensor = torch.randn(1, 3, 64, 224, 224).cuda()

    # Explicitly create a TensorDataset and DataLoader
    dummy_dataset = TensorDataset(dummy_input_tensor)
    dummy_dataloader = DataLoader(dummy_dataset, batch_size=1, shuffle=False) # Ensure batch_size=1

    trainer = pl.Trainer(
        max_epochs=10,
        devices=1,
        accelerator="gpu",
        enable_checkpointing=False,
        log_every_n_steps=1
    )

    print("🚀 Finalizing Integrated NeMo V-JEPA 2 Audit...")
    trainer.fit(model, train_dataloaders=dummy_dataloader)

    print("\n🔍 Execution Complete. Running Final Physics Audit...")
    model.cuda() # Ensure model is on GPU before manual audit
    res = model.audit_physics(dummy_input_tensor) # Use the original dummy_input_tensor for audit
    print(f"{'='*50}\n📊 VERDICT: {res['status']}\n📉 FINAL LPE: {res['lpe']:.6f}\n{'='*50}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


🚀 Finalizing Integrated NeMo V-JEPA 2 Audit...


┏━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name        ┃ Type               ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ patch_embed │ Conv3d             │  1.6 M │ train │     0 │
│ 1 │ transformer │ TransformerEncoder │  151 M │ train │     0 │
│ 2 │ predictor   │ Sequential         │  2.1 M │ train │     0 │
└───┴─────────────┴────────────────────┴────────┴───────┴───────┘

Trainable params: 154 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 154 M                                                                                                
Total estimated model params size (MB): 619                                                                        
Modules in train mode: 127                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

[NeMo W 2026-01-22 20:00:23 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
    


...

DEBUG: _get_tokens returning x shape: torch.Size([1, 6272, 1024])

DEBUG: training_step tokens shape: torch.Size([1, 6272, 1024])

DEBUG: Shape of visible_tokens before transformer: torch.Size([1, 2509, 1024])

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.



🔍 Execution Complete. Running Final Physics Audit...
DEBUG: _get_tokens input x shape: torch.Size([1, 3, 64, 224, 224])
DEBUG: _get_tokens after patch_embed x shape: torch.Size([1, 1024, 32, 14, 14])
DEBUG: _get_tokens returning x shape: torch.Size([1, 6272, 1024])
📊 VERDICT: CRITICAL
📉 FINAL LPE: 36.058586
